In [142]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from dateutil import parser

In [2]:
link = "https://www.trustpilot.com/review/www.rebtel.com"
link_ext = "?page="

In [3]:
pages = [str(i) for i in range(2,107)]
links =  [link] + [link + link_ext + i for i in pages]

In [4]:
# extract all data from all days
all_reviews = []
all_titles = []
all_replies = []
all_profiles = []
all_ratings = []


all_countries = [] #obtain from profiles
all_reads = [] #obtain from profiles
all_reviews = [] #obtain from profiles
all_usefuls = [] #obtain from profiles
all_review_times = [] #obtain from ratings

#all_reply_times = [] #obtain from tmp_reply_times
#all_likes = [] 
#all_updates = [] 
#all_genres = [] #obtain form profiles*

for link in links:
    
    html = requests.get(link).text
    soup = BeautifulSoup(html, "lxml")
    
    reviews = soup.findAll("div",{"class": 'review-body'})
    titles = soup.findAll("a",{"class": 'review-title-link'})
    profiles = soup.findAll("a",{"class": 'user-review-name-link'})
    ratings = soup.findAll("div",{"class": 'review-info clearfix'})
    tmp_reply_times = soup.findAll("div",{"class": 'company-reply'})
    replies = soup.findAll("div",{"class": 'comment'})
    
    page_data = [reviews, titles, profiles, ratings, tmp_reply_times,replies]
    all_reviews.append(page_data)

In [6]:
all_ratings = []
all_review_times = []

for page_reviews in all_reviews:
    
    page_ratings = []
    page_times = []
    
    #get ratings and time_reviews per page
    for rating_review in page_reviews[3]:
        
        rating = rating_review.contents[1].attrs['class'][1][-1]
        time = rating_review.contents[3].attrs['title']
        page_ratings.append(rating)
        page_times.append(time)
        
    all_ratings.append(page_ratings)
    all_review_times.append(page_times)


In [25]:
all_text_reviews = []
all_title_reviews = []


for page_reviews in all_reviews:
    
    page_text_reviews = []
    page_title_reviews = []
    
    #get text reviews per page
    for rating_review in page_reviews[0]:
        
        
        raw_text = rating_review.contents[0]
        text = raw_text.lstrip().rstrip()
        page_text_reviews.append(text)
        
    all_text_reviews.append(page_text_reviews)
        
    #get title reviews per page
    for rating_review in page_reviews[1]:
        
        title = rating_review.contents[0]
        page_title_reviews.append(title)
    all_title_reviews.append(page_title_reviews)

In [32]:
extended_profiles = []

for page_reviews in all_reviews:
    
    profile_links = []
    website = 'https://www.trustpilot.com/'
    page_profiles = []
    
    #get profile links
    for profile in page_reviews[2]:
        
        tmp_prof_link = website + profile.attrs['href']
        profile_links.append(tmp_prof_link)
        
    
    for link in profile_links:
    
        html = requests.get(link).text
        soup = BeautifulSoup(html, "lxml")
    
        tmp_summ = soup.find("div",{"class": 'user-summary-gender-location'})
        country = tmp_summ.contents[-2].contents[0]
    
        tmp_name = soup.find("h1",{"class": 'user-summary-info__user-name'})
        user_name = tmp_name.contents[0]
    
        tmp_rev = soup.find("span",{"class": 'user-reviews-stats__reviewcount'})
        num_rev = tmp_rev.attrs['data-count']
    
        tmp_reads = soup.find("span",{"class": 'user-reviews-stats__readcount'})
        num_reads = tmp_reads.attrs['data-count']
    
        tmp_useful = soup.find("span",{"class": 'user-reviews-stats__usefulcount'})
        num_useful = tmp_useful.attrs['data-count']
    
        tmp_genre = soup.find("div",{"class": 'user-summary-gender'})
        genre = tmp_genre.contents[0]
    
        profile_data = [country, user_name, num_rev, num_reads, num_useful, genre]
    
        page_profiles.append(profile_data)
    
    extended_profiles.append(page_profiles)

In [258]:
#Preprocessing profile data
for page_review in extended_profiles:
    for profile in page_review:
        
        profile[1] = profile[1].rstrip().lstrip()
        profile[-1] = profile[-1].rstrip().lstrip()
        tmp_country = profile[0].rstrip().lstrip()
        
        if tmp_country.count(',') is 0:
            profile[0] = tmp_country
        else:
            profile[0] = tmp_country.rsplit()[-1]
        

In [249]:
profile = extended_profiles[0][0]

In [253]:
x = profile[0]

In [256]:
x.count(',') is 10

False

In [ ]:
#Preprocessing time data
clean_time_reviews = []

for page in all_review_times:
    page_time_reviews = []
    
    for review_time in page:
        ptime = parser.parse(review_time)
        clean_time = [ptime.day,ptime.month,ptime.year,
                      ptime.hour,ptime.minute,ptime.second,
                      ptime.isoweekday(),ptime.isocalendar()[1],
                      ptime.isoformat()]
        page_time_reviews.append(clean_time)
        
    clean_time_reviews.append(page_time_reviews)

In [259]:
def flatten_data(collected_data):
    
    data = [item for sublist in collected_data for item in sublist]
    return data

ext_profiles_f =  flatten_data(extended_profiles)
title_reviews_f = flatten_data(all_title_reviews)
text_reviews_f = flatten_data(all_text_reviews)
ratings_f = flatten_data(all_ratings)
time_reviews_f = flatten_data(clean_time_reviews)

In [260]:
ext_prof_df = pd.DataFrame(ext_profiles_f,columns=['Country','Username','Reviews','Reads','Useful','Genre'])

In [261]:
title_df = pd.DataFrame(title_reviews_f,columns=['Title'])

In [262]:
review_df = pd.DataFrame(text_reviews_f,columns=['Review'])

In [263]:
rating_df = pd.DataFrame(ratings_f,columns=['Rating'])

In [264]:
time_df = pd.DataFrame(time_reviews_f,columns=['day','month','year','hour','minute',
                                           'second','dayofweek','weekofyear',
                                           'raw time'])

In [265]:
data = pd.concat([review_df, title_df, rating_df,time_df, ext_prof_df], axis=1)

In [266]:
data.columns

Index(['Review', 'Title', 'Rating', 'day', 'month', 'year', 'hour', 'minute',
       'second', 'dayofweek', 'weekofyear', 'raw time', 'Country', 'Username',
       'Reviews', 'Reads', 'Useful', 'Genre'],
      dtype='object')

In [281]:
data.to_csv('rebtel.csv',index=False)

In [ ]:
#extract profiles from all users
profile_links = []
website = 'https://www.trustpilot.com/'
page_profiles = []

#get profile links
for profile in all_reviews[0][2]:
    tmp_prof_link = website + profile.attrs['href']
    profile_links.append(tmp_prof_link) 

for link in profile_links:
    
    html = requests.get(link).text
    soup = BeautifulSoup(html, "lxml")
    
    tmp_summ = soup.find("div",{"class": 'user-summary-gender-location'})
    country = tmp_summ.contents[-2].contents[0]
    
    tmp_name = soup.find("h1",{"class": 'user-summary-info__user-name'})
    user_name = tmp_name.contents[0]
    
    tmp_rev = soup.find("span",{"class": 'user-reviews-stats__reviewcount'})
    num_rev = tmp_rev.attrs['data-count']
    
    tmp_reads = soup.find("span",{"class": 'user-reviews-stats__readcount'})
    num_reads = tmp_reads.attrs['data-count']
    
    tmp_useful = soup.find("span",{"class": 'user-reviews-stats__usefulcount'})
    num_useful = tmp_useful.attrs['data-count']
    
    tmp_genre = soup.find("div",{"class": 'user-summary-gender'})
    genre = tmp_genre.contents[0]
    
    profile_data = [country, user_name, num_rev, num_reads, num_useful, genre]
    
    page_profiles.append(profile_data)
    

In [ ]:
detailed_profiles[19]

In [ ]:
link = profile_links[0]
html = requests.get(link).text
soup = BeautifulSoup(html, "lxml")

In [ ]:
tmp_summ = soup.find("div",{"class": 'user-summary-gender-location'})
country = tmp_summ.contents[-2].contents[0]

In [ ]:
tmp_name = soup.find("h1",{"class": 'user-summary-info__user-name'})
user_name = tmp_name.contents[0]

In [ ]:
tmp_rev = soup.find("span",{"class": 'user-reviews-stats__reviewcount'})
num_rev = tmp_rev.attrs['data-count']

In [ ]:
tmp_reads = soup.find("span",{"class": 'user-reviews-stats__readcount'})
num_reads = tmp_reads.attrs['data-count']

In [ ]:
tmp_useful = soup.find("span",{"class": 'user-reviews-stats__usefulcount'})
num_useful = tmp_useful.attrs['data-count']

In [ ]:
tmp_genre = soup.find("div",{"class": 'user-summary-gender'})
genre = tmp_genre.contents[0]


In [ ]:
profile_data = [country, user_name, num_rev, num_reads, num_useful, genre]

In [ ]:
link = "https://www.trustpilot.com/review/www.rebtel.com"

html = requests.get(link).text
soup = BeautifulSoup(html, "lxml")

In [ ]:
#soup.findAll("div",{"class": 'review-body'})
soup.findAll("div",{"class": 'text useful-counter-js hidden'})

In [ ]:
#soup.find("div", {'class': 'companyinformation-wrapper clearfix'})
soup.findAll("a",{"class": 'user-review-name-link'})

In [ ]:
link = "https://www.architecture.com/FindAnArchitect/FAAPractices.aspx?display=50"

html = requests.get(link).text

"""If you do not want to use requests then you can use the following code below 
   with urllib (the snippet above). It should not cause any issue."""
soup = BeautifulSoup(html, "lxml")
res = soup.findAll("article", {"class": "listingItem"})
for r in res:
    print("Company Name: " + r.find('a').text)
    print("Address: " + r.find("div", {'class': 'address'}).text)
    print("Website: " + r.find_all("div", {'class': 'pageMeta-item'})[3].text)